# 2D Visualization of 3D Data
## Zachary Neronha, Wong Lab
### 26 October 2018, created for the purpose of simple viewing of data in two dimensions 

In [1]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib inline
from __future__ import division
init_notebook_mode(connected=True)
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username='zjneronha', api_key='PMrKGUGAet6kUZ5Rcf5E')
import brewer2mpl as cbrew

In [2]:
#Create helper functions

#function 1: return the cells within a certain location
def findRange(xPts,yPts,xRange,yRange):
    cellList = []
    for ii in range(0,np.size(xPts,0)):
        qLog = np.greater(xPts[ii,:],xRange[0]) & np.less(xPts[ii,:],xRange[1])
        pLog = np.greater(yPts[ii,:],yRange[0]) & np.less(yPts[ii,:],yRange[1])
        qLog = sum(qLog)
        pLog = sum(pLog)

        if qLog > 0 and pLog >0:
            cellList.append(ii)
    
    return cellList

#function 2: pull and export only the cells of interest 
def selExport(xPts,yPts,zPts,MyCells,pathName):
    selX = xPts[MyCells,:]
    selY = yPts[MyCells,:]
    selZ = zPts[MyCells,:]
    
    dSx = pd.DataFrame(selX)
    dSy = pd.DataFrame(selY)
    dSz = pd.DataFrame(selZ)
    
    ptsSZE = sum(sum(np.logical_not(np.isnan(selX))))
    selPTS = np.empty((ptsSZE,3))
    loopcounter = 0
    
    #convert back to direct matrix format and export
    for ii  in range(0,np.size(selX,0)):
        for jj in range(0,np.size(selX,1)):
            if np.isnan(selX[ii,jj])==0:
                selPTS[loopcounter,:] = [selX[ii,jj],selY[ii,jj],selZ[ii,jj]]
                loopcounter = loopcounter + 1
                
    dTPS = pd.DataFrame(selPTS)
    dTPS.to_csv(pathname,index=False,header = False)
    
    print("Feature Complete; File Saved!")

In [3]:
currentwell = 20
interval = (1/3) #minutes between imaging 

#import data from csv
xlsZ = pd.ExcelFile("NuclearTracking/ConvertedData/w"+repr(currentwell)+"_combineddata.xls")
dxx = pd.read_excel(xlsZ,'xStore')
dyy = pd.read_excel(xlsZ,'yStore')
dzz = pd.read_excel(xlsZ,'zStore')
xDat = np.array(dxx)
yDat = np.array(dyy)
zDat = np.array(dzz)
#cmap = cbrew.get_map('PuOr','Diverging',5).mpl_colors
#colors = [list(elem) for elem in cmap]

In [4]:
pltdata = []
for i in range(0,np.size(xDat,0)):
    # Create a trace
    trace = go.Scatter(
        name = 'Cell ' + repr(i),
        x = xDat[i,:],
        y = yDat[i,:],   
        )
    pltdata.append(trace)
data = [pltdata]

py.iplot(pltdata, filename='basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~zjneronha/0 or inside your plot.ly account where it is named 'basic-line'


/home/dbhaskar92/anaconda2/lib/python2.7/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [5]:
xRan = [350,370]
yRan = [510,540]
celL = findRange(xDat,yDat,xRan,yRan)

In [6]:
#X and Y temporal plots, frequency estimation, etc.
xred = xDat[celL,:]
yred = yDat[celL,:]
trL = np.multiply(xred,0)+1
for k in range(0,np.size(trL,1)):
    trL[:,k] = np.multiply(trL[:,k],k)

trL = np.multiply(trL,interval)

In [7]:
#plot sequentially
pltdata = []
for i in range(0,np.size(xred,0)):
    # Create a trace
    trace = go.Scatter(
        name = 'Cell ' + repr(i),
        x = trL[i,:],
        y = xred[i,:], 
        )
    pltdata.append(trace)
data = [pltdata]

py.iplot(pltdata, filename='XFREQ')


In [8]:
#plot Y frequency
pltdata = []
for i in range(0,np.size(xred,0)):
    # Create a trace
    trace = go.Scatter(
        name = 'Cell ' + repr(i),
        x = trL[i,:],
        y = yred[i,:], 
        )
    pltdata.append(trace)
data = [pltdata]

py.iplot(pltdata, filename='YFREQ')

In [9]:
pathname = "SelectedFeatures/Loop1_w"+repr(currentwell)+".csv"
selExport(xDat,yDat,zDat,celL,pathname)

Feature Complete; File Saved!
